# PRIMO 2022 ML Homework 4<br>
### Estimating the issue priority in the bug tracker

Задача: предсказать, имеет ли проблема высокий приоритет, учитывая ее описание и метаданные.<br>
Данные: взяты с сайта youtrack.jetbrains.com.<br>
Оценка качества: F1 мера

Метрика оценки качества для соревнования: F1 мера для класса 1.<br>
<br>
**Формат файла с предсказаниями<br>**
Для каждой задачи в наборе test итоговый файл с предсказаниями должен содержать два значения: id (id задачи, взятый из столбца `id` в наборе данных test) и предсказание приоритета задачи (1, если приоритет высокий, или 0, если приоритет низкий).<br>
<br>
Файл должен иметь заголовок и следующий формат:<br>
<br>
id,is_high_priority<br>
0,1<br>
1,1<br>
2,1<br>

In [75]:
import pandas as pd
import numpy as np

import warnings
import json
import operator

from tqdm import tqdm

from sklearn.model_selection import train_test_split


from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import MultinomialNB
#np.set_printoptions(precision=3)
#pd.set_option('precision', 3)



In [76]:
test_data = pd.read_csv('test.csv', index_col='id')

In [77]:
train_data = pd.read_csv('train.csv', index_col='id')

Перемешаем данные для чанков

In [ ]:
train_data = train_data.sample(frac = 1)

In [79]:
chunks = np.array_split(train_data, 4, )
chunk1 = chunks[0]
chunk2 = chunks[1]
chunk3 = chunks[2]
chunk4 = chunks[3]

chunk1

,summary,description,reporter,created,customFields,links,is_high_priority
id,,,,,,,
25-2414794,"Find action: ""Changelists: Enable changelist c...","![](image.png)\n\nThis cannot be turned on, it...","{""login"": ""vqrslogin"", ""$type"": ""User""}",1562663019192,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False
25-627054,Ctrl+Q on the suggestions dropdown in a Spring...,Create a Spring XML file and create a bean ins...,"{""login"": ""Mihai.Danila"", ""$type"": ""User""}",1383605502961,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False
25-787759,Don't lock password manager on workstation lock,In IntelliJ 13 you only had to enter you passw...,"{""login"": ""Leonard_Br\u00fcnings"", ""$type"": ""U...",1415810924197,"[{""value"": {""name"": ""Usability Problem"", ""$typ...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False
25-695053,Git plugin ignores part of hook error response...,Diff:\ncommand line:\nremote:\nremote: REJECTE...,"{""login"": ""Nikolay.Bukharev"", ""$type"": ""User""}",1399366811882,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False
25-1923361,Alternative key behavior on Find bar,What steps will reproduce the issue?\n1. Press...,"{""login"": ""thiagoalves"", ""$type"": ""User""}",1522599380935,"[{""value"": {""name"": ""Usability Problem"", ""$typ...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False
...,...,...,...,...,...,...,...
25-926265,"By default all test launchers should be ""Singl...","I think that JUnit, ScalaTest, Spec launcher s...","{""login"": ""David_Perez"", ""$type"": ""User""}",1430387117683,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False
25-768716,When you save a file automatically formatted code,"\n1.8.0_20-b26x32 Oracle Corporation, Windows ...","{""login"": ""tonycody"", ""$type"": ""User""}",1413982790072,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False
25-1979575,"Gradle project and Java 9,10,11 Modules, Intel...",I'm not using com.intellij.uiDesigner in any o...,"{""login"": ""francogpellegrini"", ""$type"": ""User""}",1525637113600,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False


In [80]:
chunk1.summary.fillna('', inplace=True)
chunk1.description.fillna('', inplace=True)

In [81]:
test_data.summary.fillna('', inplace=True)
test_data.description.fillna('', inplace=True)

In [82]:
X_text = chunk1.summary
y = chunk1.is_high_priority
X_test = test_data.summary
X_text

id
25-2414794    Find action: "Changelists: Enable changelist c...
25-627054     Ctrl+Q on the suggestions dropdown in a Spring...
25-787759       Don't lock password manager on workstation lock
25-695053     Git plugin ignores part of hook error response...
25-1923361                 Alternative key behavior on Find bar
                                    ...                        
25-926265     By default all test launchers should be "Singl...
25-768716     When you save a file automatically formatted code
25-1979575    Gradle project and Java 9,10,11 Modules, Intel...
25-563571     Preview for in-place regexp replace doesn't pr...
25-2426706                                Cannot Resolve Symbol
Name: summary, Length: 26342, dtype: object

In [83]:
y.value_counts()

False    25213
True      1129
Name: is_high_priority, dtype: int64

In [84]:
X_text_train, X_text_test, y_train, y_test = train_test_split(X_text, y,
                                                              stratify=y,
                                                              random_state=3,
                                                              test_size=0.25,
                                                              shuffle=True)

In [85]:
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en.stop_words import STOP_WORDS

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(min_df=.001, max_df=0.5,
                            token_pattern=r'[A-Za-z]{2,}',
                            stop_words=STOP_WORDS)

tfidf_vec.fit(X_text_train)

X_text_vect_sample = pd.DataFrame(tfidf_vec.transform(X_text_train).todense(),
                                  columns=tfidf_vec.get_feature_names_out())

X_text_vect_sample

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


,ability,able,abstract,accept,access,account,action,actions,actionscript,active,...,wrap,wrapped,wrapper,wrapping,wraps,wrong,wrongly,xml,yaml,youtrack
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.345395,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
19752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
19753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
19754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [87]:

class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [88]:
# https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB, MultinomialNB, ComplementNB

nb_pipeline = Pipeline(steps = [('tfidf', tfidf_vec),
                                ('to_dense', DenseTransformer()),
                                ('clf', ComplementNB())
                               ]
                       )

nb_pipeline.fit(X_text_train, y_train)

print (f'Train f1 = {f1_score(y_true=y_train, y_pred=nb_pipeline.predict(X_text_train))}')
print (f'Test f1 = {f1_score(y_true=y_test, y_pred=nb_pipeline.predict(X_text_test))}')

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Train f1 = 0.16673684210526316
Test f1 = 0.11157024793388429


c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [89]:
from sklearn.ensemble import RandomForestRegressor
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.01, .05], max_df=[.7, .8])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [('tfidf', TfidfVectorizer(min_df=params['min_df'], max_df=params['max_df'],
                                                       token_pattern=r'[A-Za-z]{2,}',
                                                       stop_words=STOP_WORDS)),
                             ('to_dense', DenseTransformer()),
                             ('clf', ComplementNB())
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_text_train, y_train)
    train_preds = pipe.predict(X_text_train)

    results.append(dict(
                        estimator=pipe,
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=pipe.predict(X_text_train)),
                        test_f1 = f1_score(y_true=y_test, y_pred=pipe.predict(X_text_test))
    ))

  0%|          | 0/4 [00:00<?, ?it/s]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
c:\users\w

In [90]:
pd.DataFrame(results).drop(columns='estimator').sort_values('test_f1').style.bar(vmin=0, vmax=1)

,parameters,train_f1,test_f1
1,"{'max_df': 0.7, 'min_df': 0.05}",0.093492,0.090253
3,"{'max_df': 0.8, 'min_df': 0.05}",0.093492,0.090253
0,"{'max_df': 0.7, 'min_df': 0.01}",0.123480,0.103476
2,"{'max_df': 0.8, 'min_df': 0.01}",0.123480,0.103476


In [91]:
json_fields = pd.json_normalize(chunk1.customFields.map(json.loads).map(lambda x: {field['name']: field['value'] for field in x}))
json_fields.set_index(chunk1.index, inplace=True)
json_fields.head()

,Affected versions,Included in builds,Available in,Type.name,Type.$type,State.name,State.$type,Assignee.login,Assignee.name,Assignee.$type,Subsystem.name,Subsystem.$type,Assignee,Subsystem
id,,,,,,,,,,,,,,
25-2414794,"[{'name': '192.5587.17', '$type': 'VersionBund...","[{'name': '193.1193', '$type': 'BuildBundleEle...",[],Bug,EnumBundleElement,Fixed,StateBundleElement,sdv,Dmitriy Smirnov,User,Version Control,OwnedBundleElement,NaN,NaN
25-627054,[],"[{'name': '132.972', '$type': 'BuildBundleElem...",[],Bug,EnumBundleElement,Fixed,StateBundleElement,yannc76,Yann Cebron,User,Frameworks. Spring,OwnedBundleElement,NaN,NaN
25-787759,[],[],[],Usability Problem,EnumBundleElement,Fixed,StateBundleElement,develar,Vladimir Krivosheev,User,Core. Password Management,OwnedBundleElement,NaN,NaN
25-695053,[],[],NaN,Bug,EnumBundleElement,Shelved,StateBundleElement,Ivan.Semenov,Ivan Semenov,User,Version Control. Git,OwnedBundleElement,NaN,NaN
25-1923361,[],[],NaN,Usability Problem,EnumBundleElement,Shelved,StateBundleElement,Vassiliy.Kudryashov,Vassiliy Kudryashov,User,"User Interface. Find, Replace, Usages",OwnedBundleElement,NaN,NaN


In [92]:
json_fields.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26342 entries, 25-2414794 to 25-2426706
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Affected versions   26342 non-null  object 
 1   Included in builds  26342 non-null  object 
 2   Available in        7869 non-null   object 
 3   Type.name           26342 non-null  object 
 4   Type.$type          26342 non-null  object 
 5   State.name          26342 non-null  object 
 6   State.$type         26342 non-null  object 
 7   Assignee.login      24129 non-null  object 
 8   Assignee.name       24129 non-null  object 
 9   Assignee.$type      24129 non-null  object 
 10  Subsystem.name      20729 non-null  object 
 11  Subsystem.$type     20729 non-null  object 
 12  Assignee            0 non-null      float64
 13  Subsystem           0 non-null      float64
dtypes: float64(2), object(12)
memory usage: 3.0+ MB


In [93]:
json_columns = ['Type.name' , 'Subsystem.name']

In [94]:
chunk1 = chunk1.join(json_fields[json_columns], how='outer', lsuffix='_left')

chunk1['reporter_name'] = chunk1.reporter.map(json.loads).map(operator.itemgetter('login'))
chunk1['Year'] = chunk1.created.apply(lambda c: pd.to_datetime(c, unit='ms').year)
chunk1['Month'] = chunk1.created.apply(lambda c: pd.to_datetime(c, unit='ms').month)
chunk1['Day'] = chunk1.created.apply(lambda c: pd.to_datetime(c, unit='ms').day)
chunk1['Hour'] = chunk1.created.apply(lambda c: pd.to_datetime(c, unit='ms').hour)

In [95]:
chunk1

,summary,description,reporter,created,customFields,links,is_high_priority,Type.name,Subsystem.name,reporter_name,Year,Month,Day,Hour
id,,,,,,,,,,,,,,
25-2414794,"Find action: ""Changelists: Enable changelist c...","![](image.png)\n\nThis cannot be turned on, it...","{""login"": ""vqrslogin"", ""$type"": ""User""}",1562663019192,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False,Bug,Version Control,vqrslogin,2019,7,9,9
25-627054,Ctrl+Q on the suggestions dropdown in a Spring...,Create a Spring XML file and create a bean ins...,"{""login"": ""Mihai.Danila"", ""$type"": ""User""}",1383605502961,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False,Bug,Frameworks. Spring,Mihai.Danila,2013,11,4,22
25-787759,Don't lock password manager on workstation lock,In IntelliJ 13 you only had to enter you passw...,"{""login"": ""Leonard_Br\u00fcnings"", ""$type"": ""U...",1415810924197,"[{""value"": {""name"": ""Usability Problem"", ""$typ...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False,Usability Problem,Core. Password Management,Leonard_Brünings,2014,11,12,16
25-695053,Git plugin ignores part of hook error response...,Diff:\ncommand line:\nremote:\nremote: REJECTE...,"{""login"": ""Nikolay.Bukharev"", ""$type"": ""User""}",1399366811882,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False,Bug,Version Control. Git,Nikolay.Bukharev,2014,5,6,9
25-1923361,Alternative key behavior on Find bar,What steps will reproduce the issue?\n1. Press...,"{""login"": ""thiagoalves"", ""$type"": ""User""}",1522599380935,"[{""value"": {""name"": ""Usability Problem"", ""$typ...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False,Usability Problem,"User Interface. Find, Replace, Usages",thiagoalves,2018,4,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-926265,"By default all test launchers should be ""Singl...","I think that JUnit, ScalaTest, Spec launcher s...","{""login"": ""David_Perez"", ""$type"": ""User""}",1430387117683,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False,Bug,Java. Tests. JUnit,David_Perez,2015,4,30,9
25-768716,When you save a file automatically formatted code,"\n1.8.0_20-b26x32 Oracle Corporation, Windows ...","{""login"": ""tonycody"", ""$type"": ""User""}",1413982790072,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False,Bug,NaN,tonycody,2014,10,22,12
25-1979575,"Gradle project and Java 9,10,11 Modules, Intel...",I'm not using com.intellij.uiDesigner in any o...,"{""login"": ""francogpellegrini"", ""$type"": ""User""}",1525637113600,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""...",False,Bug,Java. GUI Designer,francogpellegrini,2018,5,6,20


In [96]:
columns = ['summary', 'reporter_name', 'Year','Month', 'Day', 'Hour'] +  json_columns
#columns = ['summary'] +  json_columns

X = chunk1[columns]
y = chunk1['is_high_priority']

In [97]:
X.Year.value_counts()

2012    2928
2013    2799
2018    2722
2017    2705
2011    2566
2014    2412
2016    2368
2015    2242
2010    2142
2019    2081
2009    1377
Name: Year, dtype: int64

In [98]:
from sklearn_pandas import DataFrameMapper

preprocessor = DataFrameMapper([('summary', TfidfVectorizer(min_df=params['min_df'], max_df=params['max_df'],
                                                       token_pattern=r'[A-Za-z]{2,}',
                                                       stop_words=STOP_WORDS)),
                                (['Type.name'], OneHotEncoder(sparse=False, handle_unknown='ignore')),
                                #(['reporter_name'], ...),
                                (['Subsystem.name'], OneHotEncoder(sparse=False, handle_unknown='ignore')),
                                ('Year', None),
                                ('Month', None),
                                ('Day', None),
                                ('Hour', None),
                                ],
                               input_df=True,
                               df_out=True)


preprocessor.fit_transform(X.sample(5))

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Functi

,summary_aborted,summary_auto,summary_block,summary_case,summary_cherry,summary_clean,summary_complete,summary_constructor,summary_current,summary_directories,...,Type.name_x0_Usability Problem,Subsystem.name_x0_Core. Project Settings,Subsystem.name_x0_Java. Code Completion,Subsystem.name_x0_Java. Refactorings,Subsystem.name_x0_Version Control. Git,Subsystem.name_x0_nan,Year,Month,Day,Hour
id,,,,,,,,,,,,,,,,,,,,,
25-1859609,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.447214,...,0.0,1.0,0.0,0.0,0.0,0.0,2018,2,5,15
25-2502097,0.447214,0.447214,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.0,0.0,0.0,1.0,0.0,2019,9,20,10
25-343457,0.000000,0.000000,0.301511,0.000000,0.000000,0.000000,0.301511,0.000000,0.301511,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,2011,9,3,14
25-2502942,0.000000,0.000000,0.000000,0.339992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.0,0.0,1.0,0.0,0.0,2019,9,22,6
25-620778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.387757,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,2013,10,16,23


In [99]:
pipe = make_pipeline(preprocessor, ComplementNB())

pipe.fit(X, y)

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Pipeline(steps=[('dataframemapper',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[('summary',
                                            TfidfVectorizer(max_df=0.8,
                                                            min_df=0.05,
                                                            stop_words={"'d",
                                                                        "'ll",
                                                                        "'m",
                                                                        "'re",
                                                                        "'s",
                                                                        "'ve",
                                                                        'a',
                                                                        'about',
                                                                        'ab

In [100]:
probabilities = pipe.predict_proba(X)[:, 1]

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [101]:
probabilities

array([0.49538952, 0.63002408, 0.63204387, ..., 0.66767607, 0.43715025,
       0.40234515])

In [102]:
threshold = 0.6

In [103]:
f1_score(y_true=y, y_pred=probabilities > threshold)

0.16227912008654888